# Data Prepreprocessing
In this, we are going to preprocess data. The following conventions are followed:
1. An example labeled as normal will be labeled as 1.
2. An example labeled as attack will be labeled as 0.
3. The columns that feature categoritcal data will be divided into multiple columns using one-hot vector

In [21]:
## INCLUDING LIBRARIES
import pandas as pd
import time
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [22]:
#LOADING THE DATASET
dataset1 = pd.read_csv('attack.csv')
dataset2 = pd.read_csv('normal.csv')
# dataset1 = dataset1.iloc[:150000,:]
# dataset2 = dataset2.iloc[:150000,:]
dataset = pd.concat([dataset1, dataset2], ignore_index=True) #concatinating two dataset
dataset = dataset.sample(frac=1).reset_index(drop=True)  #shuffling
dataset.head()

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,duration,protocol,Plength,flag,Mlength,HoP,LifeTime,MsgType,DSN,Sno,...,land,Tmode,Neighbors,Hflow,AvgFlow,Lflow,AvgHopCount,failedConnection,Failed Rate,Label
0,0.004359,ICMP,92,-1,28,-1,-1,-1,-1,6,...,2,0,13,6354,1830.000000,290,0.214960,4502,62.632165,attack
1,0.001493,ICMP,92,-1,20,-1,-1,-1,-1,13,...,2,0,16,7097,1926.312500,25,0.197682,4733,59.647133,attack
2,0.001059,ICMP,92,-1,28,-1,-1,-1,-1,15,...,2,0,13,6398,1832.076923,291,0.214430,4500,62.595632,attack
3,0.000050,AODV,76,0,20,-1,-1,Route Error,0,5,...,2,1,15,6634,2044.600000,265,0.194770,4848,61.499429,normal
4,0.001158,ICMP,92,-1,20,-1,-1,-1,-1,3,...,2,0,15,6634,2044.600000,265,0.194770,4848,61.499429,attack


## Removing unwanted or unnecessary features

In [ ]:
dataset = dataset.drop(['flag','DSN','LifeTime'], axis=1)

### Label encoding of Categorical data

In [23]:
le = LabelEncoder()
protocol_labels = le.fit_transform(dataset['protocol'])
protocol_mappings = {index: label for index, label in 
                  enumerate(le.classes_)}
protocol_mappings

{0: 'AODV', 1: 'ICMP', 2: 'UDP'}

In [24]:
dataset['protocol'] = protocol_labels

In [25]:
pro_ohe = OneHotEncoder()
pro_feature_arr = pro_ohe.fit_transform(dataset[['protocol']]).toarray()
pro_feature_labels = list(le.classes_)
pro_features = pd.DataFrame(pro_feature_arr, 
                            columns=pro_feature_labels)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [26]:
dataset = pd.concat([dataset, pro_features], axis=1)

In [27]:
dataset = dataset.drop(['protocol'], axis=1)
dataset.head()

,duration,Plength,Mlength,HoP,MsgType,Sno,Sindex,land,Tmode,Neighbors,Hflow,AvgFlow,Lflow,AvgHopCount,failedConnection,Failed Rate,Label,AODV,ICMP,UDP
0,0.004359,92,28,-1,-1,6,2486,2,0,13,6354,1830.000000,290,0.214960,4502,62.632165,attack,0.0,1.0,0.0
1,0.001493,92,20,-1,-1,13,2994,2,0,16,7097,1926.312500,25,0.197682,4733,59.647133,attack,0.0,1.0,0.0
2,0.001059,92,28,-1,-1,15,2037,2,0,13,6398,1832.076923,291,0.214430,4500,62.595632,attack,0.0,1.0,0.0
3,0.000050,76,20,-1,Route Error,5,937,2,1,15,6634,2044.600000,265,0.194770,4848,61.499429,normal,1.0,0.0,0.0
4,0.001158,92,20,-1,-1,3,707,2,0,15,6634,2044.600000,265,0.194770,4848,61.499429,attack,0.0,1.0,0.0


In [28]:
#X = dataset.iloc[:, :-1].values
le = LabelEncoder()
msgtype_labels = le.fit_transform(dataset['MsgType'])
genre_mappings = {index: label for index, label in 
                  enumerate(le.classes_)}
genre_mappings

{0: '-1',
 1: 'Route Error',
 2: 'Route Reply',
 3: 'Route Reply Acknowledgment',
 4: 'Route Request'}

In [29]:
dataset['labels'] = msgtype_labels

In [30]:
msg_ohe = OneHotEncoder()
msg_feature_arr = msg_ohe.fit_transform(dataset[['MsgType']]).toarray()
msg_feature_labels = list(le.classes_)
msg_features = pd.DataFrame(msg_feature_arr, 
                            columns=msg_feature_labels)

In [31]:
dataset_transformed = pd.concat([dataset, msg_features], axis=1)

In [32]:
dataset_transformed.head()

,duration,Plength,Mlength,HoP,MsgType,Sno,Sindex,land,Tmode,Neighbors,...,Label,AODV,ICMP,UDP,labels,-1,Route Error,Route Reply,Route Reply Acknowledgment,Route Request
0,0.004359,92,28,-1,-1,6,2486,2,0,13,...,attack,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0
1,0.001493,92,20,-1,-1,13,2994,2,0,16,...,attack,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0
2,0.001059,92,28,-1,-1,15,2037,2,0,13,...,attack,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0
3,0.000050,76,20,-1,Route Error,5,937,2,1,15,...,normal,1.0,0.0,0.0,1,0.0,1.0,0.0,0.0,0.0
4,0.001158,92,20,-1,-1,3,707,2,0,15,...,attack,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0


In [33]:
dataset_transformed = dataset_transformed.rename(columns={"-1": "Unkown MsgType"})
dataset_transformed = dataset_transformed.drop(['MsgType'], axis=1)

In [34]:
dataset_transformed.head()

,duration,Plength,Mlength,HoP,Sno,Sindex,land,Tmode,Neighbors,Hflow,...,Label,AODV,ICMP,UDP,labels,Unkown MsgType,Route Error,Route Reply,Route Reply Acknowledgment,Route Request
0,0.004359,92,28,-1,6,2486,2,0,13,6354,...,attack,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0
1,0.001493,92,20,-1,13,2994,2,0,16,7097,...,attack,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0
2,0.001059,92,28,-1,15,2037,2,0,13,6398,...,attack,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0
3,0.000050,76,20,-1,5,937,2,1,15,6634,...,normal,1.0,0.0,0.0,1,0.0,1.0,0.0,0.0,0.0
4,0.001158,92,20,-1,3,707,2,0,15,6634,...,attack,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0


In [35]:
Y = pd.DataFrame(dataset_transformed['Label'])
Y['Label'] = le.fit_transform(Y['Label'])
X = dataset_transformed.drop(['Label'], axis=1)
X.to_csv('X-data.csv',index = False)
Y.to_csv('Y-data.csv',index = False)
X.head()

,duration,Plength,Mlength,HoP,Sno,Sindex,land,Tmode,Neighbors,Hflow,...,Failed Rate,AODV,ICMP,UDP,labels,Unkown MsgType,Route Error,Route Reply,Route Reply Acknowledgment,Route Request
0,0.004359,92,28,-1,6,2486,2,0,13,6354,...,62.632165,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0
1,0.001493,92,20,-1,13,2994,2,0,16,7097,...,59.647133,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0
2,0.001059,92,28,-1,15,2037,2,0,13,6398,...,62.595632,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0
3,0.000050,76,20,-1,5,937,2,1,15,6634,...,61.499429,1.0,0.0,0.0,1,0.0,1.0,0.0,0.0,0.0
4,0.001158,92,20,-1,3,707,2,0,15,6634,...,61.499429,0.0,1.0,0.0,0,1.0,0.0,0.0,0.0,0.0


In [36]:
#print(Y.head())
X_ = X.iloc[:, :].values
Y_ = Y.iloc[:, :].values

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X_,Y_, test_size=0.2)

In [38]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [19]:
X_train

array([[-0.15991066,  1.41421356,  0.57735027, -1.        ,  1.72038703,
         1.69968293,  1.        , -1.73205081,  0.98787834,  0.30289762,
         0.78969531, -0.43420589,  0.74704142,  0.6164976 , -1.12487518,
        -1.73205081,  1.73205081,  0.        , -1.50755672,  1.73205081,
        -0.57735027, -1.        ],
       [ 1.66416891,  0.        ,  0.57735027,  1.        , -0.40961596,
        -0.83796312, -1.        ,  0.57735027,  0.10976426,  1.44698519,
        -0.43643874, -0.65018395,  0.93209861,  0.31866607,  0.53569496,
         0.57735027, -0.57735027,  0.        ,  0.90453403, -0.57735027,
        -0.57735027,  1.        ],
       [-0.56032977, -1.41421356, -1.73205081, -1.        , -0.57346234,
        -0.56800078,  1.        ,  0.57735027, -1.6464639 , -1.25139987,
        -1.42632974, -0.64118486, -1.60225136, -1.70877423,  1.36092028,
         0.57735027, -0.57735027,  0.        , -0.30151134, -0.57735027,
         1.73205081, -1.        ],
       [-0.94392847